In [67]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

# Goal: reproduce nanoGPT tuto from Karpathy
**plan:**
- first implement a word embedder
    - [x] I already got the data(Shakespeare books)
    - [x] read that data
    - [ ] tokenize the data
    - [ ] create an embedder layer 
    - [ ] generator: generate new text: how do I recover a letter based on the output of my network? 

- modify the code to support an attention mechanism


In [68]:
import random 

# printing first 1000 characters of input.txt
with open("input.txt", 'r') as file:
    text = file.read()
    print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [69]:
# since the idea is to create a character prediction program I guess we should tokenize the characters, not words 

# grab all the different characters in our text
# from karpathy: - sorted - print by joining - vocab_size (should be useful later) 
characters = sorted(list(set(text)))
vocab_size = len(characters)
print(''.join(characters))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [70]:
# I guess this map can account for an embedder 
# from karpathy: -encode -decode, I was doing it for a single character or integer, it should be for a list of such. plus, for the decoder we should output a string, not a list of characters
stoi = {elem:i for i,elem in enumerate(characters)} # stoi: string to index 
itos = {i:elem for i,elem in enumerate(characters)} # 
encode = lambda element: [stoi[elem] for elem in element]
decode = lambda element: ''.join([itos[elem] for elem in element])
print(stoi)
print(encode('hell$$o asd s'))
print(decode(encode('hell$$o asd s')))

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
[46, 43, 50, 50, 3, 3, 53, 1, 39, 57, 42, 1, 57]
hell$$o asd s


So Andrej goes on to say that this is heavily based on Bigram.
What is Bigram? 

In [71]:
# - work with batches
# - work with blocks 
import torch

data = torch.tensor(encode(text))
print(data.shape)
print(data[:1000])


torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57

In [72]:
# then we split our data: 90% for train and 10% for validation 

n = int(len(data)*0.9) # 
train_data = data[:n]
val_data = data[n:]
print(train_data.shape)
print(val_data.shape)


torch.Size([1003854])
torch.Size([111540])


In [73]:
# we have to partition our data
# and work in batches
# meaning we will have a collection of `batch_size` random numbers (i.e positions in the text array) and for each of these positions we will
# get a `block`

block_size = 8
batch_size = 4

def get_batch(stage):
    data = train_data if stage=='train' else val_data 
    positions = torch.randint(len(data)-block_size,(batch_size,))
    x = [data[pos:pos+block_size] for pos in positions]
    y = [data[pos+1:pos+block_size+1] for pos in positions]
    # now we need to stack everything
    x = torch.stack(x)
    y = torch.stack(y)
    return x,y


x = train_data[:block_size]
y = train_data[1:block_size+1]


# for i in range(block_size):
#     foo = x[:i+1]
#     target = y[i]
#     print(f'for the context{foo} be have the target {target}')

xb, yb = get_batch('train')

print('input:')
print(xb.shape)
print(xb)

print('target')
print(yb.shape)
print(yb)

print('----')


for i in range(batch_size):
    for j in range(block_size):
        print(f'when input is {xb[i][:j+1]} the target is: {yb[i][j]}')




input:
torch.Size([4, 8])
tensor([[52, 43, 47, 45, 46, 40, 53, 59],
        [39, 58,  1, 58, 46, 43,  1, 58],
        [61,  1, 47, 52,  1, 51, 63,  1],
        [ 1, 43, 51, 40, 56, 39, 41, 43]])
target
torch.Size([4, 8])
tensor([[43, 47, 45, 46, 40, 53, 59, 56],
        [58,  1, 58, 46, 43,  1, 58, 47],
        [ 1, 47, 52,  1, 51, 63,  1, 40],
        [43, 51, 40, 56, 39, 41, 43, 51]])
----
when input is tensor([52]) the target is: 43
when input is tensor([52, 43]) the target is: 47
when input is tensor([52, 43, 47]) the target is: 45
when input is tensor([52, 43, 47, 45]) the target is: 46
when input is tensor([52, 43, 47, 45, 46]) the target is: 40
when input is tensor([52, 43, 47, 45, 46, 40]) the target is: 53
when input is tensor([52, 43, 47, 45, 46, 40, 53]) the target is: 59
when input is tensor([52, 43, 47, 45, 46, 40, 53, 59]) the target is: 56
when input is tensor([39]) the target is: 58
when input is tensor([39, 58]) the target is: 1
when input is tensor([39, 58,  1]) the t

In [104]:
# now that I have my data and targets, we should be able to define an architecture to process them
import torch
import torch.nn as nn 
from torch.nn import functional as F
torch.manual_seed(1337)

# ----- hyperparameters -----------
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300 
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200 
n_embd = 32
# ---------------------------------

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # this matrix serves as a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # ATTENTION: we now create a level of interaction
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):

        # idx: batch of blocks
        # for each entry of `stoi = idx[][]`, we will retrieve the stio-th row of self.token_embedding_table
        tok_emb = self.token_embedding_table(idx) # (B,T,C) = (B = batch_size, time = block_size, channels = vocal_size)
        # targets dimension = (B, T, C) as well
        logits = self.lm_head(tok_emb) # (B,T, vocab_size)


        if targets is None:
            loss = None
        else: 
            # mine: loss = F.cross_entropy(logits.view(logits.shape[0],logits.shape[2],logits.shape[1]),targets)
            # karpathy: 
            B,T,C = logits.shape
            logits  = logits.view(B*T,C)
            targets = targets.view(-1)  
            loss = F.cross_entropy(logits,targets)

        return logits, loss
    
    # if I call self(idx) then the `forward` function will be applied to `idx`    
    def generate(self, idx, max_new_tokens):
        
        for i in range(max_new_tokens):            
            logits,loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits,dim=1)
            idx_next = torch.multinomial(probs,num_samples=1)
            idx = torch.cat((idx,idx_next),dim=1)

        return idx

m = BigramLanguageModel()
logits, loss  = m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx= torch.zeros((1,1),dtype=torch.long), max_new_tokens=500)[0].tolist()))



torch.Size([256, 65])
tensor(4.6927, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3!dcbf?pGXepydZJSrF$Jrqt!:wwWSzPNxbjPiD&Q!a;yNt$Kr$o-gC$WSjJqfBKBySKtSKpwNNfyl&w:q-jluBatD$Lj;?yzyUca!UQ!vrpxZQgC-hlkq,ptKqHoiX-jjeLJ &slERj KUsBOL!mpJO!zLg'wNfqHAMgq'hZCWhu.W.IBcP 
RFJ&DEs,nw?pxE?xjNHHVxJ&D&vWWToiERJFuszPyZaNw$
EQJMgzaveDDIoiMl&sMHkzdRptRCPVjwW.RSVMjs-bgRkzrBTEa!!oP fRSxq.PLboTMkX'DUYepxIBFAYuxKXe.jeh
sa!3MGFrSjuM:wX!?BTMl!.?,M:bQzPHpYfN!Cbo'MmtDxBkDD3SBjyFdmY'DOqkWeRjlxyJB-bVbfd&


In [75]:
# learned a new word in english: pluck out 

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)


In [76]:
batch_size = 32
for steps in range(500):
    xb, yb = get_batch('train')

    # evaluate 
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.156820774078369


In [77]:
print(decode(m.generate(idx= torch.zeros((1,1),dtype=torch.long), max_new_tokens=500)[0].tolist()))


GXzzWChe awNulI3CjysuzbCKuy,$p$zCHocLEbQB-ULTXlfNfMGMgOkzmytiZYhiwf OOgU-R;Adow?YdffivOKh:C:CbfSFixQGqSU?q-KH:ChKHAUzd$su-MyiA'Stxju
l,OMmRSAUAAHJukSP CeVPEN:CEaog
lgCe
ju-IowOymnQ!z'!oiAL;AcKRF.?cgXQNmawNX&HUN$..szv'huwajJh CjuQ?wpMly;$s,i-!UBco!BShiTIsuhzLkSP?,BLfl!.?oUzXL&y:Q'rgOTHZSYfjJVTXEoGs
YDU'a!mObjx?Y-INNWlt'hs!BBDAcS.
Osr,H-tKfXrUzX3daD YQL Wczc
GzkHoiGTaP :3.?Er'QbfT:Aen;IUxqep;ly'qKzrsXE!paNQz:iy:CvUA'zolve:C -
OKvLYCj;P nAyFM :Ts!uDnEa'brDU, -taHxrepJXaKiiBTR?YhOrd3fxB&yiM
D3y
iBuJ


# the mathematical trick 

In [78]:
# define the toy model 

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [98]:
# we want x[b,t]=mean_{t'<=t} x[b,t']

xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [99]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
# print(xbow2[1])
torch.allclose(xbow,xbow2,rtol=1e-3)
# [torch.allclose(xbow[i], xbow2[i]) for i in range(B)]

True

the use of `tril` from pytorch

In [83]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [102]:
# version using softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3,rtol=1e-3)

True

## some preliminaries